# SkyWater Inverter Design using GPlugins

The goal of this notebook is to demonstrate the integration of the `sky130nm` electronic PDK within a gdsfactory-based schematic driven layout. Broadly, we will explore:

* A little introduction at setting up the existing open-source electronic EDA development flow with `xschem` using the latest `sky130nm` library
* Exporting that schematic into a SPICE model, which is used to schematic-driven-layout using gdsfactory.
* Example inverter design and layout between the toolsets.

## Using the `gdsfactory-sky130nm` Repository

The goal of this notebook is to show that analogue simulation and layout can be performed together within a `gdsfactory` environment.

```
pip install gplugins[hdl21,sky130]
```

In [ ]:
import hdl21 as h
import gdsfactory as gf
import gplugins.hdl21 as gph
import sky130
import sky130_hdl21

from bokeh.io import output_notebook
from gdsfactory.config import rich_output
from gplugins.schematic_editor import SchematicEditor

gf.config.rich_output()

%env BOKEH_ALLOW_WS_ORIGIN=*

output_notebook()

In [ ]:
# sky130.cells
# sky130.cells["sky130_fd_pr__cap_vpp_02p4x04p6_m1m2_noshield"]()

In [ ]:
@h.module
class SkyInv:
    """ An inverter, demonstrating using PDK modules """

    # Create some IO
    i, o, VDD, VSS = h.Ports(4)

    p = sky130_hdl21.Sky130MosParams(w=1,l=1)

    # And create some transistors!
    ps = sky130_hdl21.primitives.PMOS_1p8V_STD(p)(d=o, g=i, s=VDD, b=VDD)
    ns = sky130_hdl21.primitives.NMOS_1p8V_STD(p)(d=VSS, g=i, s=o, b=VSS)

### Manually editing the `SPICE`-generated `gdsfactory` component YAML

It is important to know that with the SPICE-generated YAML, we cannot actually create a layout on its own. This is because the SPICE models do not exactly directly map to layout instances. SPICE models can represent performance corners for the same device, with multiple temperature or yield quality variations. As such, we need to assign the corresponding gds we want to layout for our specific schematic model.

In [ ]:
example_inverter_manual_yaml = gph.generate_raw_yaml_from_module(
    SkyInv
)
print(example_inverter_manual_yaml)

In [ ]:


example_inverter_manual_yaml = """
connections:
  ns,d: ps,d
  ns,g: ps,g
  ps,d: ns,d
  ps,g: ns,g
instances:
  ns:
    component: sky130_fd_pr__nfet_01v8
    info: {}
    settings:
      As: int((nf+2)/2) * w/nf * 0.29
      ad: int((nf+1)/2) * w/nf * 0.29
      l: UNIT_1
      m: UNIT_1
      mult: UNIT_1
      nf: UNIT_1
      nrd: 0.29 / w
      nrs: 0.29 / w
      pd: 2*int((nf+1)/2) * (w/nf + 0.29)
      ports:
        b: VSS
        d: o
        g: i
        s: VSS
      ps: 2*int((nf+2)/2) * (w/nf + 0.29)
      sa: UNIT_0
      sb: UNIT_0
      sd: UNIT_0
      w: UNIT_1
  ps:
    component: sky130_fd_pr__pfet_01v8
    info: {}
    settings:
      As: int((nf+2)/2) * w/nf * 0.29
      ad: int((nf+1)/2) * w/nf * 0.29
      l: UNIT_1
      m: UNIT_1
      mult: UNIT_1
      nf: UNIT_1
      nrd: 0.29 / w
      nrs: 0.29 / w
      pd: 2*int((nf+1)/2) * (w/nf + 0.29)
      ports:
        b: VDD
        d: o
        g: i
        s: VDD
      ps: 2*int((nf+2)/2) * (w/nf + 0.29)
      sa: UNIT_0
      sb: UNIT_0
      sd: UNIT_0
      w: UNIT_1
name: SkyInv
ports:
  VDD: ps,s
  VSS: ns,s
  i: ps,g
  o: ps,d
"""
with open("data/sky130nm/example_inverter_manual.schem.yaml", 'w') as file:
        file.write(example_inverter_manual_yaml)

### Automatically mapping layout instances to the YAML - Inverter

In [ ]:
example_inverter_schematic_editor = gph.hdl21_module_to_schematic_editor(
    module=SkyInv,
    yaml_schematic_file_name="data/sky130nm/example_inverter_auto.schem.yaml",
)
example_inverter_schematic_editor.visualize()

In [ ]:
example_inverter_layout = "data/sky130nm/example_inverter_auto.layout.yaml"
example_inverter_schematic_editor.instantiate_layout(example_inverter_layout, default_router="get_bundle", default_cross_section="xs_metal1")
c = gf.read.from_yaml(example_inverter_layout)
c.plot()

### More Advanced Example - R2R DAC


An example of using `hdl21.ExternalModule`s representing an implementation technology/ PDK
in a parametric resistive DAC generator mapping to a gdsfactory implementation.

In [ ]:
import sys
from typing import Optional

import hdl21 as h

PdkResistor = sky130_hdl21.ress["GEN_PO"]
Nch = sky130_hdl21.primitives.NMOS_1p8V_STD
Pch = sky130_hdl21.primitives.PMOS_1p8V_STD


@h.paramclass
class RLadderParams:
    """# Resistor Ladder Parameters

    Note the use of the `hdl21.Instantiable` type for the unit resistor element.
    This generally means "something we can make an `Instance` of" - most commonly a `Module`.

    Here we will be using a combination of an `ExternalModule` and its parameter-values,
    in something like:

    ```python
    RLadderParams(res=PdkResistor(w=4 * µ, l=10 * µ))
    ```
    """

    nseg = h.Param(dtype=int, desc="Number of segments")
    res = h.Param(dtype=h.Instantiable, desc="Unit resistor, with params applied")


@h.generator
def rladder(params: RLadderParams) -> h.Module:
    """# Resistor Ladder Generator"""

    @h.module
    class RLadder:
        # IO
        top = h.Inout()
        bot = h.Inout()
        taps = h.Output(width=params.nseg - 1)

        # Create concatenations for the P and N sides of each resistor
        nsides = h.Concat(bot, taps)
        psides = h.Concat(taps, top)

        # And create our unit resistor array
        runits = params.nseg * params.res(p=psides, n=nsides)

    return RLadder


@h.paramclass
class PassGateParams:
    """# Pass Gate Parameters

    See the commentary on `RLadderParams` above regarding the use of `hdl21.Instantiable`,
    which here serves as the parameter type for each transistor. It will generally be used like:

    ```python
    PassGateParams(
        nmos=Nch(PdkMosParams(l=1 * n)),
        pmos=Pch(PdkMosParams(l=1 * n)),
    )
    ```

    Both `nmos` and `pmos` parameters are `Optional`, which means they can be set to the Python built-in `None` value.
    If either is `None`, its "half" of the pass gate will be omitted.
    Setting *both* to `None` will cause a generator exception.
    """

    nmos = h.Param(dtype=Optional[h.Instantiable], desc="NMOS. Disabled if None.")
    pmos = h.Param(dtype=Optional[h.Instantiable], desc="PMOS. Disabled if None")


@h.generator
def passgate(params: PassGateParams) -> h.Module:
    """# Pass Gate Generator"""
    if params.nmos is None and params.pmos is None:
        raise RuntimeError("A pass gate needs at least *one* transistor!")

    @h.module
    class PassGate:
        source = h.Inout()
        drain = h.Inout()

    if params.pmos is not None:
        PassGate.VDD = h.Inout()
        PassGate.en_b = h.Input()
        PassGate.PSW = params.pmos(
            d=PassGate.drain, s=PassGate.source, g=PassGate.en_b, b=PassGate.VDD
        )

    if params.nmos is not None:
        PassGate.VSS = h.Inout()
        PassGate.en = h.Input()
        PassGate.NSW = params.nmos(
            d=PassGate.drain, s=PassGate.source, g=PassGate.en, b=PassGate.VSS
        )

    return PassGate


@h.generator
def mux(params: PassGateParams) -> h.Module:
    """# Pass-Gate Analog Mux Generator"""

    @h.module
    class Mux:
        sourceA = h.Input()
        sourceB = h.Input()
        out = h.Output()
        ctrl = h.Input()
        ctrl_b = h.Input()

    aconns, bconns = (
        dict(source=Mux.sourceA, drain=Mux.out),
        dict(source=Mux.sourceB, drain=Mux.out),
    )
    if params.pmos is not None:
        Mux.VDD = h.Inout()
        aconns["VDD"] = Mux.VDD
        aconns["en_b"] = Mux.ctrl_b
        bconns["VDD"] = Mux.VDD
        bconns["en_b"] = Mux.ctrl
    if params.nmos is not None:
        Mux.VSS = h.Inout()
        aconns["VSS"] = Mux.VSS
        aconns["en"] = Mux.ctrl
        bconns["VSS"] = Mux.VSS
        bconns["en"] = Mux.ctrl_b
    Mux.passgate_a = passgate(params)(**aconns)
    Mux.passgate_b = passgate(params)(**bconns)
    return Mux


@h.paramclass
class MuxTreeParams:
    """# Mux Tree Parameters"""

    nbit = h.Param(dtype=int, desc="Number of bits")
    mux_params = h.Param(dtype=PassGateParams, desc="Parameters for the MUX generator")


@h.generator
def mux_tree(params: MuxTreeParams) -> h.Module:
    """Binary Mux Tree Generator"""

    n_inputs = 2**params.nbit
    p_ctrl = params.mux_params.nmos is not None
    n_ctrl = params.mux_params.pmos is not None

    # Base module
    @h.module
    class MuxTree:
        out = h.Output()
        v_in = h.Input(width=n_inputs)
        ctrl = h.Input(width=params.nbit)
        ctrl_b = h.Input(width=params.nbit)

    base_mux_conns = dict()
    if p_ctrl:
        MuxTree.VSS = h.Inout()
        base_mux_conns["VSS"] = MuxTree.VSS
    if n_ctrl:
        MuxTree.VDD = h.Inout()
        base_mux_conns["VDD"] = MuxTree.VDD

    # Build the MUX tree layer by layer
    curr_input = MuxTree.v_in
    for layer in range(params.nbit - 1, -1, -1):
        layer_mux_conns = base_mux_conns.copy()
        layer_mux_conns["ctrl"] = MuxTree.ctrl[layer]
        layer_mux_conns["ctrl_b"] = MuxTree.ctrl_b[layer]
        if layer != 0:
            curr_output = MuxTree.add(
                name=f"sig_{layer}", val=h.Signal(width=2**layer)
            )
        else:
            curr_output = MuxTree.out
        for mux_idx in range(2**layer):
            mux_conns = layer_mux_conns.copy()
            mux_conns["sourceA"] = curr_input[2 * mux_idx]
            mux_conns["sourceB"] = curr_input[2 * mux_idx + 1]
            mux_conns["out"] = curr_output[mux_idx]
            MuxTree.add(
                name=f"mux_{layer}_{mux_idx}", val=mux(params.mux_params)(**mux_conns)
            )
        curr_input = curr_output
    return MuxTree


"""Main function, generating an `rladder` and `mux_tree` and netlisting each."""

# Create parameter values for each of our top-level generators
rparams = RLadderParams(
    nseg=15,
    res=PdkResistor(),
)
mparams = MuxTreeParams(
    nbit=4,
    mux_params=PassGateParams(
        nmos=Nch(),
        pmos=Pch(),
    ),
)

# Netlist in a handful of formats
duts = [rladder(rparams)]
# h.netlist(duts, sys.stdout, fmt="verilog")
# h.netlist(duts, sys.stdout, fmt="spectre")
# h.netlist(duts, sys.stdout, fmt="spice")
# h.netlist(duts, sys.stdout, fmt="xyce")

In [ ]:
example_resistor_ladder_schematic_editor = gph.hdl21_module_to_schematic_editor(
    module=rladder(rparams),
    yaml_schematic_file_name="data/sky130nm/rladder.schem.yaml",
)
example_resistor_ladder_schematic_editor.visualize()

In [ ]:
example_inverter_layout = "data/sky130nm/example_inverter_auto.layout.yaml"
example_inverter_schematic_editor.instantiate_layout(example_inverter_layout, default_router="get_bundle", default_cross_section="xs_metal1")
c = gf.read.from_yaml(example_inverter_layout)
c.plot()

## (WIP, future integrations) Setting up the required tools: `xschem`, `volare` and the `sky130nm` PDKs

I will say early on, you would benefit from working in a UNIX or Debian Linux environment. Most EDA tools either proprietary or open-source only work within these operating systems or a Docker environment of these operating systems.

There are multiple ways to get started with the environment, you could, for example:
1. Work within a [IIC-OSIC-TOOLS docker environment](https://github.com/iic-jku/IIC-OSIC-TOOLS). There is also a nice [OSIC-multitool project](https://github.com/iic-jku/osic-multitool) specifically for SKY130nm designs.
2. Install these tools natively in your operating system. You could follow one of these tutorials, for example:
    * https://www.engineerwikis.com/wikis/installation-of-xschem
    * https://www.youtube.com/watch?v=jXmmxO8WG8s&t=7s

This tutorial assumes the tools have already been installed in a local operating system. We will use some design files generated by these toolsets as guided by this [OSIC-Multitool example](https://github.com/iic-jku/osic-multitool/tree/main/example/ana). Note we recommend cloning the `gplugins` repository.

In [ ]:
import pathlib
from gplugins.spice import parse_netlist as spice

Our example files are under the directory of `gplugins/notebooks/data`, let's extract our SPICE declaration:

In [ ]:
def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            text = file.read()
        return text
    except FileNotFoundError:
        print("File not found. Please provide a valid file path.")
        return None

Let's list the files we're going to be reading into:

In [ ]:
inverter_spice_file = pathlib.Path("data") / "sky130nm" / "inv.sch"

Let's extract our raw spice netlist. An important aspect to understand is that unfortunately, every SPICE tool developed their own file format. So in this sense, netlist parsing function is implemented according to the type of spice toolset that has generated this netlist.

We aim to support:

- `xschem`
- `lumerical?`

In [ ]:
inverter_spice_text = read_file(inverter_spice_file)
inverter_spice_text

In [ ]:
spice.netlist_text

In [ ]:
elements, _, _ = spice.parse_netlist_and_extract_elements(netlist_text=spice.netlist_text, spice_type="lumerical")
elements

So we can use our `netlist` parsing function to convert this to a compatible netlist for gdsfactory plugins into the extracted elements and the extracted connections:

In [ ]:
inverter_netlist_elements, inverter_netlist_connections, _ = spice.parse_netlist_and_extract_elements(netlist_text=inverter_spice_text, spice_type="xschem")
inverter_netlist_elements, inverter_netlist_connections, _

### Automated schematic-driven-layout

We have now extracted our spice elements and our connectivity. Let's explore what we have there: